In [2]:
!pip install tensorflow-macos tensorflow-metal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 MB 34.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 42.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 29.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.32.1 requires protobuf<6.0,>=5.0, but you have protobuf 4.25.7 which is incompatible.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.1.0 which is incompatible.
grpcio-tools 1.67.1 requires protobuf<6.0dev,>=5

In [5]:
import os
import numpy as np
import pandas as pd
from itertools import product
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Dense, Input

# ─── Configuration ─────────────────────────────────────────────────────────
FOLDS_DIR = "cv_folds"
FEATURE_COLS = [
    "PTS_8_game_avg", "PTS_8_game_slope",
    "REB_8_game_avg", "REB_8_game_slope",
    "AST_8_game_avg", "AST_8_game_slope",
    "opp_E_OFF_RATING", "opp_E_DEF_RATING",
    "opp_E_NET_RATING", "opp_REB_PCT"
]
TARGET_COLS = ["PTS", "REB", "AST"]
N_JOBS = 4

# ─── Hyperparameter grid ──────────────────────────────────────────────────
param_grid = {
    "filters": [16, 32],
    "kernel_size": [3, 5],
    "epochs": [10, 20]
}

# ─── CNN builder ────────────────────────────────────────────────────────────
def make_cnn(input_shape, n_outputs, filters, kernel_size):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters, kernel_size, activation="relu", padding="same"),
        Conv1D(filters//2, kernel_size, activation="relu", padding="same"),
        GlobalAveragePooling1D(),
        Dense(16, activation="relu"),
        Dense(n_outputs)
    ])
    model.compile(optimizer="adam", loss="mse")
    return model

# ─── Single-fold runner (returns per-target metrics) ───────────────────────
def run_cnn_fold(fold_idx, hp):
    train = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_train.csv", parse_dates=["GAME_DATE"])
    test  = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_test.csv",  parse_dates=["GAME_DATE"])
    X_train = train[FEATURE_COLS].values; y_train = train[TARGET_COLS].values
    X_test  = test[FEATURE_COLS].values;  y_test  = test[TARGET_COLS].values

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train); X_test = scaler.transform(X_test)
    X_train = X_train[:, None, :]; X_test = X_test[:, None, :]

    model = make_cnn(input_shape=X_train.shape[1:], n_outputs=len(TARGET_COLS),
                     filters=hp["filters"], kernel_size=hp["kernel_size"])
    model.fit(X_train, y_train, epochs=hp["epochs"], batch_size=32, verbose=0)
    preds = model.predict(X_test, verbose=0)

    metrics = {"fold": fold_idx, **hp}
    for idx, target in enumerate(TARGET_COLS):
        y_true = y_test[:, idx]
        y_pred = preds[:, idx]
        metrics[f"{target}_rmse"] = np.sqrt(mean_squared_error(y_true, y_pred))
        metrics[f"{target}_mae"]  = mean_absolute_error(y_true, y_pred)
        metrics[f"{target}_r2"]   = r2_score(y_true, y_pred)
    return metrics

# ─── Run grid search with per-fold, per-target metrics ────────────────────
all_results = []
for filters, kernel_size, epochs in product(param_grid["filters"],
                                           param_grid["kernel_size"],
                                           param_grid["epochs"]):
    hp = {"filters": filters, "kernel_size": kernel_size, "epochs": epochs}
    fold_metrics = Parallel(n_jobs=N_JOBS)(
        delayed(run_cnn_fold)(fold, hp) for fold in range(1, 11)
    )
    all_results.extend(fold_metrics)

# ─── Compile and display results ────────────────────────────────────────────
final_df = pd.DataFrame(all_results)
# Order columns: hyperparams, fold, then metrics
cols = ["filters","kernel_size","epochs","fold"] + [c for c in final_df.columns if c not in ("filters","kernel_size","epochs","fold")]
final_df = final_df[cols]
print(final_df.to_string(index=False))


2025-05-04 20:52:56.834032: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-05-04 20:52:56.834061: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-04 20:52:56.834065: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-04 20:52:56.834082: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-04 20:52:56.834095: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-05-04 20:52:57.779037: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-05-04 20:52:57.779083: I metal_plugin/src/device/metal_device.cc:296] syste

 filters  kernel_size  epochs  fold  PTS_rmse   PTS_mae    PTS_r2  REB_rmse  REB_mae        REB_r2  AST_rmse  AST_mae    AST_r2
      16            3      10     1  9.611333  7.244059 -0.117950  3.423504 2.753523 -1.431966e-02  2.973699 2.340698 -0.003769
      16            3      10     2  8.708920  6.963093 -0.000377  3.542408 2.778251 -3.695488e-06  2.919996 2.196014 -0.002597
      16            3      10     3  8.494537  6.865648 -0.001455  3.560344 2.644088 -1.148939e-03  2.717552 2.141672 -0.000079
      16            3      10     4  8.927136  7.136915 -0.000958  3.650653 2.776712 -9.570122e-04  2.807475 2.157648 -0.001164
      16            3      10     5  8.790373  7.139827 -0.000157  3.689078 2.834845 -1.993418e-03  2.851491 2.201478 -0.001219
      16            3      10     6  9.084348  7.189756 -0.002423  3.624707 2.768007 -2.264977e-06  2.829979 2.210521 -0.000313
      16            3      10     7  8.737908  7.062966 -0.000444  3.614328 2.794944 -8.821487e-06  2.76